<a href="https://colab.research.google.com/github/edolubis/AB-Testing/blob/main/AB_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proses AB Testing
1. Menyusun hipotesis
2. Menentukan matriks keputusan & analisis power

analisis power digunakan untuk melihat seberapa kuat kekuatan ujinya,
sedangkan matriks keputusan juga digunakan dalam bisnis.

3. baru melakukan AB Testing Random
4. Pengujian berdasarkan statistik

Ho (hipotesis null) = adalah hipotesis yang akan ditolak

**MATRIKS KEPUTUSAN**
Pentingnya matriks keputusan DIAWAL supaya mencegah p-value peeking, sehingga testnya tidak dapat DIMAINKAN.

**Eksperimen Pre-Analysis**
jumlah sampel harus MEMILIKI statistical power yang cukup.

**SIGINIFICANCE LEVEL VS POWER APA BEDANYA?** DAN APA PERSAMAANYA?
Persamaanya adalah sama-sama mendeteksi perubahan, yang bedanya adalah
Siginificance Level itu dengan **ASUMSI PERUBAHANNYA TIDAK ADA** (5%)
Power itu dengan **ASUMSI PERUBAHAN DARI DETECABLE ITU ADA** (RULE OF THUMB 8%)


kemudian ada Minimum Detectable Effect
minimum angka yang mau kita lihat dari testingnya


LANGKAH SELANJUTNYA MELAKUKAN PEMILIHAN MODEL
melakukan pemilihan model & uji statistiknya


Pada case ini business metric nya adalah Conversion Rate (dalam bentuk probabilitas, dimana distribusinya **DISTRIBUSI NORMAL**)

Formulasi Hipotesis
Ho = Rata2 conversion rate kedua banner sama saja Pa = Pb
H1 = Rata2 conversion rate kedua banner beda Pa tidak samadengan Pb

karena distribusinya normal, maka menggunakan **proportion Z-test two tail.**

CONTOH STUDI KASUS

1. Ada Test Split
2. Conversion Rate
3. Relative Effect
4. Num Traffic

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.read_csv('ab.csv')
dataframe.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [ ]:
data = dataframe.copy().drop(columns=['user_id', 'landing_page'])
data

,timestamp,group,converted
0,2017-01-21 22:11:48.556739,control,0
1,2017-01-12 08:01:45.159739,control,0
2,2017-01-11 16:55:06.154213,treatment,0
3,2017-01-08 18:28:03.143765,treatment,0
4,2017-01-21 01:52:26.210827,control,1
...,...,...,...
294473,2017-01-03 22:28:38.630509,control,0
294474,2017-01-12 00:51:57.078372,control,0
294475,2017-01-22 11:45:03.439544,control,0
294476,2017-01-15 01:20:28.957438,control,0


In [ ]:
# ekstrak tipe data waktu
data['date'] = pd.to_datetime(data['timestamp']).dt.date

In [ ]:
# urutkan berdasarkan waktu
data.sort_values(by='timestamp')

,timestamp,group,converted,date
131228,2017-01-02 13:42:05.378582,treatment,0,2017-01-02
184884,2017-01-02 13:42:15.234051,control,0,2017-01-02
83878,2017-01-02 13:42:21.786186,control,0,2017-01-02
102717,2017-01-02 13:42:26.640581,treatment,0,2017-01-02
158789,2017-01-02 13:42:27.851110,treatment,0,2017-01-02
...,...,...,...,...
153305,2017-01-24 13:41:18.869978,treatment,0,2017-01-24
47535,2017-01-24 13:41:19.152664,control,0,2017-01-24
157787,2017-01-24 13:41:44.097174,treatment,0,2017-01-24
179072,2017-01-24 13:41:52.604673,control,0,2017-01-24


In [ ]:
abtest = data[['group', 'converted', 'date']]
abtest

,group,converted,date
0,control,0,2017-01-21
1,control,0,2017-01-12
2,treatment,0,2017-01-11
3,treatment,0,2017-01-08
4,control,1,2017-01-21
...,...,...,...
294473,control,0,2017-01-03
294474,control,0,2017-01-12
294475,control,0,2017-01-22
294476,control,0,2017-01-15


In [ ]:
# mengambil hanya nilai berupa waktu, count and sum dari converted di control & treatment group
abtest_value = abtest.groupby(['date','group']).agg(['count','sum']).unstack().reset_index().values

In [ ]:
abtest_value

array([[datetime.date(2017, 1, 2), 2894, 2889, 362, 348],
       [datetime.date(2017, 1, 3), 6684, 6710, 760, 762],
       [datetime.date(2017, 1, 4), 6655, 6629, 810, 773],
       [datetime.date(2017, 1, 5), 6538, 6586, 801, 758],
       [datetime.date(2017, 1, 6), 6684, 6844, 778, 848],
       [datetime.date(2017, 1, 7), 6678, 6703, 805, 776],
       [datetime.date(2017, 1, 8), 6769, 6795, 805, 821],
       [datetime.date(2017, 1, 9), 6725, 6714, 806, 794],
       [datetime.date(2017, 1, 10), 6745, 6778, 763, 858],
       [datetime.date(2017, 1, 11), 6787, 6766, 808, 785],
       [datetime.date(2017, 1, 12), 6615, 6707, 809, 820],
       [datetime.date(2017, 1, 13), 6649, 6589, 779, 734],
       [datetime.date(2017, 1, 14), 6635, 6694, 842, 799],
       [datetime.date(2017, 1, 15), 6809, 6640, 823, 757],
       [datetime.date(2017, 1, 16), 6683, 6644, 816, 795],
       [datetime.date(2017, 1, 17), 6691, 6631, 818, 844],
       [datetime.date(2017, 1, 18), 6578, 6707, 820, 835],
     

In [ ]:
# membuat dataframe baru
ab_test = pd.DataFrame(abtest_value, columns =['date_abtest', 'control_visit', 'treatment_visit', 'control_converted', 'treatment_converted'])

In [ ]:
abtest_df = ab_test.set_index('date_abtest').cumsum()

In [ ]:
abtest_df.head()

,control_visit,treatment_visit,control_converted,treatment_converted
date_abtest,,,,
2017-01-02,2894,2889,362,348
2017-01-03,9578,9599,1122,1110
2017-01-04,16233,16228,1932,1883
2017-01-05,22771,22814,2733,2641
2017-01-06,29455,29658,3511,3489


In [ ]:
# import libraries

from statsmodels.stats.proportion import power_proportions_2indep, test_proportions_2indep
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
abtest_df['control_converted_rate'] = abtest_df['control_converted'] / abtest_df['control_visit']
abtest_df['treatment_converted_rate'] = abtest_df['treatment_converted'] / abtest_df['treatment_visit']
abtest_df

,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate
date_abtest,,,,,,
2017-01-02,2894,2889,362,348,0.125086,0.120457
2017-01-03,9578,9599,1122,1110,0.117143,0.115637
2017-01-04,16233,16228,1932,1883,0.119017,0.116034
2017-01-05,22771,22814,2733,2641,0.120021,0.115762
2017-01-06,29455,29658,3511,3489,0.119199,0.117641
2017-01-07,36133,36361,4316,4265,0.119448,0.117296
2017-01-08,42902,43156,5121,5086,0.119365,0.117852
2017-01-09,49627,49870,5927,5880,0.119431,0.117907
2017-01-10,56372,56648,6690,6738,0.118676,0.118945


In [ ]:
def p_value(abtest_df):

  result = test_proportions_2indep(count1=abtest_df['treatment_converted'],
                                       nobs1=abtest_df['treatment_visit'],
                                       count2=abtest_df['control_converted'],
                                       nobs2=abtest_df['control_visit'],
                                       alternative='two-sided')
  return result[1]

In [ ]:
def power(abtest_df):

  test_split = abtest_df['treatment_visit'] / (abtest_df['treatment_visit'] + abtest_df['control_visit'])
  relative_effect = (abtest_df['treatment_converted_rate'] - abtest_df['control_converted_rate']) / abtest_df['control_converted_rate']

  power_result = power_proportions_2indep(diff=abtest_df['control_converted_rate'] * relative_effect,
                                               prop2=abtest_df['control_converted_rate'],
                                               nobs1=abtest_df['treatment_visit'],
                                               ratio=(1-test_split)/test_split,
                                               alpha=0.05,
                                               alternative='two-sided',
                                               return_results=False)
  return power_result

In [ ]:
abtest_df['p_value'] = abtest_df.apply(p_value, axis=1)
abtest_df['power'] = abtest_df.apply(power, axis=1)

In [ ]:
abtest_df.head()

,control_visit,treatment_visit,control_converted,treatment_converted,control_converted_rate,treatment_converted_rate,p_value,power
date_abtest,,,,,,,,
2017-01-02,2894,2889,362,348,0.125086,0.120457,0.592186,0.083522
2017-01-03,9578,9599,1122,1110,0.117143,0.115637,0.745064,0.062217
2017-01-04,16233,16228,1932,1883,0.119017,0.116034,0.404180,0.132753
2017-01-05,22771,22814,2733,2641,0.120021,0.115762,0.158646,0.291715
2017-01-06,29455,29658,3511,3489,0.119199,0.117641,0.557845,0.090324


In [ ]:
abtest_df.to_excel('final_abtest.xlsx')